In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import os
import time

start_time = time.time() #Compute time of execution

SOC_URL = "./actorMovies.csv"
df = pd.read_csv(SOC_URL, sep=";")


for i in range(df["Movies"].count()):
    df["Movies"][i] = df["Movies"][i].split("|")

df.head(25)

total = 0
for i in range(df["Movies"].count()):
    total += len(df["Movies"][i])
print(total)

10247


In [2]:
df["Movies"][18]

['Faces', 'Faces']

We have duplicated movies in some actors

In [3]:
for i in range(df["Movies"].count()):
    df["Movies"][i] = list(set(df["Movies"][i])) #Can't use pandas.unique() cos type of ["Movies"][i] is -> list   
df["Movies"][18]

['Faces']

In [4]:
movies = df.iloc[:,1].copy() #We copy the actors series
#movies = movies.to_frame() #Convert it to a dataframe
nodes = pd.DataFrame(columns=['Id','Movie'])
id = 0
for i in range(len(movies)):
    for j in range(len(movies[i])):
        nodes = nodes.append({"Id":id,"Movie":movies[i][j]}, ignore_index=True)
        id+=1
        
print(len(nodes))

10071


In [5]:
#We eliminate the duplicate movies
nodes_list = nodes
nodes_list = nodes_list["Movie"].unique()
nodes_list = pd.DataFrame({'Movie': nodes_list[:]})
nodes_list.rename(columns={0:"Id"})
nodes_list.index.name = "Id"
nodes_list.head()
nodes_list.to_csv("nodes_list.csv")
print("--- %s seconds ---" % (time.time() - start_time))
nodes_list.tail()

--- 32.0767548084259 seconds ---


,Movie
Id,
1891,Omagh
1892,The Great St Trinian's Train Robbery
1893,Hullabaloo
1894,Wasabi
1895,Hawks


In [ ]:
edges = pd.DataFrame(columns=["Source","Target","Weight","Type"])

#is_empty = lambda x,y: edges.loc[(edges["Source"] == x) & (edges["Target"] == y)].count().all() == 0
is_empty = lambda x,y,edges: edges.loc[(edges["Source"] == x) & (edges["Target"] == y)].count()[0] == 0

get_index = lambda movie_name : nodes_list.loc[nodes_list["Movie"] == movie_name, "Movie"].index[0]

def add_weight(x,y,edges):
    edges.loc[(edges["Source"] == x) & (edges["Target"] == y),"Weight"] += 1
    
def get_edges(edges):
    for i in range(df["Movies"].count()):
        for j in range(len(df["Movies"][i])):
        #Dont go over the full list -> quicker
        #why? -> last element connected in previous iter
            for k in range(j+1,len(df["Movies"][i])): #Dont add equal edges or already added
                source = get_index(df["Movies"][i][j])
                target = get_index(df["Movies"][i][k])
                if(source != target):
                    if(not is_empty(source,target,edges)):
                        add_weight(source,target,edges)
                    elif(not is_empty(target,source,edges)): #Si ya hay uno en j->k, le sumamos uno
                        add_weight(target,source, edges)
                    else:
                    #elif(is_empty(source,target) and is_empty(target,source)): #Si ambos son vacios, añadimos la arista
                        edges = edges.append({"Source":source,"Target":target,"Weight":1,"Type":"Undirected"}, ignore_index=True)

    return edges
edges = get_edges(edges)
print("--- %s seconds ---" % (time.time() - start_time))
len(edges)
edges.tail()

In [ ]:
edges.rename(columns={0:"Id"})
edges.index.name = "Id"
edges_test = edges.copy()

In [ ]:
edges_test.drop_duplicates(subset=["Source","Target"], inplace=True)
print(len(edges))
len(edges_test)

In [ ]:
edges_test.to_csv("edges_list.csv")
print("--- %s seconds ---" % (time.time() - start_time))